In [1]:
import os
import folium
import numpy as np
import pandas as pd
print(folium.__version__)

0.10.1


In [2]:
import geopandas as gpd


assert 'naturalearth_lowres' in gpd.datasets.available
datapath = gpd.datasets.get_path('naturalearth_lowres')
gdf = gpd.read_file(datapath)

In [16]:
new_data = pd.read_csv("Inch.csv")
new_data = new_data.drop('Unnamed: 0',axis=1)

In [ ]:
# load GeoJSON
with open('Boundaries - ZIP Codes.geojson', 'r') as jsonFile:
    data_ = json.load(jsonFile)
tmp = data_

# remove ZIP codes not in our dataset
geozips = []
A=[]
for i in range(len(new_data["Zip"].unique())):
    A.append(int((tmp['features'][i]['properties']['zip'])))
    if A[i] in (new_data["Zip"].unique()):
        tmp['features'][i]['properties']['zip'] = int((tmp['features'][i]['properties']['zip']))
        geozips.append(tmp['features'][i])
        
# creating new JSON object
new_json = dict.fromkeys(['type','features'])
new_json['type'] = 'FeatureCollection'
new_json['features'] = geozips

# save JSON object as updated-file
open("time_updated-file.json", "w").write(
    json.dumps(new_json, sort_keys=True, indent=4, separators=(',', ': '))
)

In [5]:
n_periods, n_sample = 8, 5
datetime_index = pd.date_range('2010-01-01',"2018-01-01", freq='A-JAN')
dt_index_epochs = datetime_index.astype(int) // 10**9
dt_index = dt_index_epochs.astype('U10')

dt_index

Index(['1264896000', '1296432000', '1327968000', '1359590400', '1391126400',
       '1422662400', '1454198400', '1485820800'],
      dtype='object')

In [7]:
styledata = {}

for ZIP in new_data.index:
    df = pd.DataFrame(
        {'color': np.random.normal(size=n_periods),
         'opacity': np.random.normal(size=n_periods)},
        index=dt_index
    )
    df = df.cumsum()
    df.sample(n_sample, replace=False).sort_index()
    styledata[ZIP] = df

In [8]:
max_color, min_color, max_opacity, min_opacity = 0, 0, 0, 0

for country, data in styledata.items():
    max_color = max(max_color, data['color'].max())
    min_color = min(max_color, data['color'].min())
    max_opacity = max(max_color, data['opacity'].max())
    max_opacity = min(max_color, data['opacity'].max())

In [9]:
from branca.colormap import linear

cmap = linear.PuRd_09.scale(min_color, max_color)


def norm(x):
    return (x - x.min()) / (x.max() - x.min())


for country, data in styledata.items():
    data['color'] = data['color'].apply(cmap)
    data['opacity'] = norm(data['opacity'])

In [10]:
styledict = {
    str(country): data.to_dict(orient='index') for
    country, data in styledata.items()
}

In [11]:
styledict = {int(k):v for k,v in styledict.items()}

In [12]:
styledict[60601]

{'1264896000': {'color': '#dbc7e1', 'opacity': 0.0},
 '1296432000': {'color': '#f6f3f9', 'opacity': 0.053464671027783905},
 '1327968000': {'color': '#f7f4f9', 'opacity': 0.20681646007468146},
 '1359590400': {'color': '#f7f4f9', 'opacity': 0.3955830531433227},
 '1391126400': {'color': '#f7f4f9', 'opacity': 0.2984029086250362},
 '1422662400': {'color': '#f7f4f9', 'opacity': 0.48965255888348164},
 '1454198400': {'color': '#f4f1f8', 'opacity': 0.6982410744217699},
 '1485820800': {'color': '#efeaf4', 'opacity': 1.0}}

In [22]:
def create_map(table, zips, mapped_feature, add_text = ''):
    # reading of the updated GeoJSON file
    geo = r'time_updated-file.json'
    # initiating a Folium map with Chicago's longitude and latitude
    coord = [41.88270035458986, -87.63663755997726]
    m = folium.Map(location = coord, tiles='cartodbpositron',zoom_start = 10)
    # creating a choropleth map
    g = TimeSliderChoropleth(geo,styledict=styledict,).add_to(m)
    m.choropleth(
        geo_data = geo,
        fill_opacity = 0.7,
        line_opacity = 0.2,
        data = table,
        # refers to which key within the GeoJSON to map the ZIP code to
        key_on = 'feature.properties.zip',
        # first element contains location information, second element contains feature of interest
        columns = [zips, mapped_feature],
        fill_color = 'BuPu',
        legend_name = (' ').join(mapped_feature.split('_')).title() + ' Number of facilites ' + add_text + ' Across Chigaco'
    )
    folium.LayerControl().add_to(m)
    # save map with filename based on the feature of interest
    m.save(outfile = mapped_feature + '_map.html')
    return(m)

In [23]:
create_map(new_data,"Zip","Violation risk")